<a href="https://colab.research.google.com/github/Nikhilsai-123/dailyassesment/blob/main/data_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
movieId,movieName,year
1,Parmanu,2018
2,URI,2019
3,rrr,2022
4,Bahubali,2015
5,tanhaji,2020
reviews
movieId,movieInfo,reviews
1,,435
2,,492
3,,444
4,,880
5,,722
6,,888
7,,220
8,,350
9,,619
10,,226

In [1]:
%%writefile movies.csv
movieId,movieName,year
1,Parmanu,2018
2,URI,2019
3,rrr,2022
4,Bahubali,2015
5,tanhaji,2020

Writing movies.csv


In [2]:
%%writefile reviews.csv
movieId,movieInfo,reviews
1,,435
2,,492
3,,444
4,,880
5,,722
6,,888
7,,220
8,,350
9,,619
10,,226

Writing reviews.csv


In [3]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=4dee3f80e48b98d176aff756afe2c77131923cdf3b6e71bb5d5e685505a43d84
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import concat,col,max,min,sum

In [5]:
spark=SparkSession.builder.appName('nikhil5').getOrCreate()

In [56]:
df=spark.read.option('header',True).csv('movies.csv')
df1=spark.read.option('header',True).csv('reviews.csv')
df.show()

+-------+---------+----+
|movieId|movieName|year|
+-------+---------+----+
|      1|  Parmanu|2018|
|      2|      URI|2019|
|      3|      rrr|2022|
|      4| Bahubali|2015|
|      5|  tanhaji|2020|
+-------+---------+----+



In [ ]:
# 1.Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins
# 2.  Fill movieInfo Column by concating movieName & year column (output should be dataframe).  Hint - try expr method
# 3. Find movie with max reviews (output should be string)
# 4. Find total reviews (output should be int)

In [30]:
#1. Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins
mv=df.join(df1,df.movieId==df1.movieId,'left')
df.join(df1,df.movieId==df1.movieId,'inner').show()
df.join(df1,df.movieId==df1.movieId,'right').show()

+-------+---------+----+-------+---------+-------+
|movieId|movieName|year|movieId|movieInfo|reviews|
+-------+---------+----+-------+---------+-------+
|      1|  Parmanu|2018|      1|     NULL|    435|
|      2|      URI|2019|      2|     NULL|    492|
|      3|      rrr|2022|      3|     NULL|    444|
|      4| Bahubali|2015|      4|     NULL|    880|
|      5|  tanhaji|2020|      5|     NULL|    722|
+-------+---------+----+-------+---------+-------+

+-------+---------+----+-------+---------+-------+
|movieId|movieName|year|movieId|movieInfo|reviews|
+-------+---------+----+-------+---------+-------+
|      1|  Parmanu|2018|      1|     NULL|    435|
|      2|      URI|2019|      2|     NULL|    492|
|      3|      rrr|2022|      3|     NULL|    444|
|      4| Bahubali|2015|      4|     NULL|    880|
|      5|  tanhaji|2020|      5|     NULL|    722|
|   NULL|     NULL|NULL|      6|     NULL|    888|
|   NULL|     NULL|NULL|      7|     NULL|    220|
|   NULL|     NULL|NULL|      

In [44]:
# 2.  Fill movieInfo Column by concating movieName & year column (output should be dataframe).  Hint - try expr method

mv.withColumn('movieInfo',concat(col('movieName'),lit(' '),col('year'))).show()
mv.withColumn('movieInfo',expr('concat(movieName,year)')).show()

+-------+---------+----+-------+-------------+-------+
|movieId|movieName|year|movieId|    movieInfo|reviews|
+-------+---------+----+-------+-------------+-------+
|      1|  Parmanu|2018|      1| Parmanu 2018|    435|
|      2|      URI|2019|      2|     URI 2019|    492|
|      3|      rrr|2022|      3|     rrr 2022|    444|
|      4| Bahubali|2015|      4|Bahubali 2015|    880|
|      5|  tanhaji|2020|      5| tanhaji 2020|    722|
+-------+---------+----+-------+-------------+-------+

+-------+---------+----+-------+------------+-------+
|movieId|movieName|year|movieId|   movieInfo|reviews|
+-------+---------+----+-------+------------+-------+
|      1|  Parmanu|2018|      1| Parmanu2018|    435|
|      2|      URI|2019|      2|     URI2019|    492|
|      3|      rrr|2022|      3|     rrr2022|    444|
|      4| Bahubali|2015|      4|Bahubali2015|    880|
|      5|  tanhaji|2020|      5| tanhaji2020|    722|
+-------+---------+----+-------+------------+-------+



In [51]:
# 3. Find movie with max reviews (output should be string)
mv.select('movieName').orderBy('reviews',ascending=False).collect()[0][0]


'Bahubali'

In [62]:
# 4. Find total reviews (output should be int)
df1.select(sum('reviews')).withColumnRenamed('sum(reviews)','count_reviews').collect()[0][0]

5276.0